In [4]:
import os 
import cv2
import numpy as np

categories = ['PNEUMONIA', 'NORMAL']
img_size = 150
def images_manipulation(data_dir):
    data = [] 
    for category in categories: 
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_image = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_image, class_num])
    return np.array(data, dtype=object)

test_data = images_manipulation('content/chest_xray/test')
train_data = images_manipulation('content/chest_xray/train')


In [6]:
print(test_data[1])

[array([[ 43,  40, 155, ...,  35,  41,  52],
       [155, 176, 168, ...,  68,  44,  27],
       [174, 174, 171, ..., 103,  45,  31],
       ...,
       [ 60,  33,  19, ...,  13,  10,  30],
       [ 64,  32,  19, ...,  11,  11,  22],
       [ 64,  33,  19, ...,  11,  22,  29]], dtype=uint8)
 0]
